In [ ]:
from pycromanager import Bridge, Acquisition, multi_d_acquisition_events, Dataset
import numpy as np
import pandas as pd
import os
import shutil
import math
import tifffile
import matplotlib.pyplot as plt
from matplotlib import cm
from skimage import io, draw, morphology, filters, measure, transform, segmentation
from scipy import spatial
from scipy.ndimage import distance_transform_edt, center_of_mass
from dmdUtils import *
from imgUtils import *

from IPython.display import clear_output

try:
    s1.close()
    s1 = initialize_led_connection()
except:
    s1 = initialize_led_connection()

bridge = Bridge()
core = bridge.get_core()


In [ ]:
# manually focus the dmd at the TIRF plane

core.set_config("-TIRF", "REFLECT-488LP")
core.set_exposure(200)
v_650(s1, 40000)
mickey_mouse()


In [ ]:
# Snap mickey mouse image for record keeping

core.snap_image()
tagged_image = core.get_tagged_image()
h, w = tagged_image.tags['Height'], tagged_image.tags['Width']
pixels = np.reshape(tagged_image.pix, newshape=[h, w])

metadata_dict = tagged_image.tags
metadata_dict['timestamp'] = str(time.time())
metadata_dict['stage_x_pos'] = str(core.get_x_position())
metadata_dict['stage_y_pos'] = str(core.get_y_position())
core.set_config("Focus", "TIPFSOffset")
core.get_position()
metadata_dict['ti_zpos'] = core.get_position()
core.set_config("Focus", "TIZDrive")
core.get_position()
metadata_dict['pfs_value'] = core.get_position()

txt_timestamp = time.strftime('%Y%m%d_%H%M%S', time.localtime(time.time()))

image_dest = f"C:\\Data\\Jason\\mickey_records\\mickey_{txt_timestamp}.tif"
tifffile.imwrite(image_dest, pixels, metadata = metadata_dict)
    
pattern = np.zeros((600,800), dtype = np.uint8)
np_to_dmd(pattern)

In [ ]:
# figure out coordinate mapping between dmd pixels and image pixels
# this can be used to estimate the affine transform

print(core.get_position())
core.set_config("-TIRF", "REFLECT-488LP")
core.set_exposure(200)

v_650(s1, 60000)

pattern = np.zeros((600,800), dtype = np.uint8)
np_to_dmd(pattern)
time.sleep(1)
core.snap_image()
tagged_image = core.get_tagged_image()
h, w = tagged_image.tags['Height'], tagged_image.tags['Width']
bg_pixels = np.reshape(tagged_image.pix, newshape=[h, w])

recorded_peaks = []
for dmd_coord_pair in [[150, 150], [450, 450], [450, 150], [150, 450]]:

    pattern = np.zeros((600,800), dtype = np.uint8)
    x, y = dmd_coord_pair
    pattern[y-10:y+11,x-10:x+11] = 1
    np_to_dmd(pattern)
    time.sleep(1)
    
    core.snap_image()
    tagged_image = core.get_tagged_image()
    h, w = tagged_image.tags['Height'], tagged_image.tags['Width']
    pixels = np.reshape(tagged_image.pix, newshape=[h, w])
    
    
    smth_pix = filters.gaussian(pixels.astype(float) - bg_pixels.astype(float), 5, preserve_range = True)
    ypeak, xpeak = np.unravel_index(np.argmax(smth_pix), shape = (h,w))
    recorded_peaks.append(xpeak)
    recorded_peaks.append(ypeak)
    
    plt.figure(figsize = (8,8))
    plt.imshow(pixels)
    plt.scatter(xpeak, ypeak, s = 500, facecolor = 'None', ec = 'k')
    print(xpeak, ypeak)
    plt.show()
    
recorded_values = open('C:\\Data\\Jason\\SCRIPTS\\calibration_coordinates.txt', 'w')
for val in recorded_peaks:
    recorded_values.write(str(val) + '\n')
recorded_values.close()

pattern = np.zeros((600,800), dtype = np.uint8)
np_to_dmd(pattern)
v_650(s1, 0)



x1, y1, x2, y2, x3, y3, x4, y4 = 150, 150, 450, 450, 450, 150, 150, 450
src = np.array([x1, y1, x2, y2, x3, y3, x4, y4]).reshape((4,2))

recorded_values = open('C:\\Data\\Jason\\SCRIPTS\\calibration_coordinates.txt', 'r')
x1, y1, x2, y2, x3, y3, x4, y4 = [int(i.strip('\n')) for i in recorded_values.readlines()]
dst = np.array([x1, y1, x2, y2, x3, y3, x4, y4]).reshape((4,2))

dmd_tform = tf.estimate_transform('affine', src, dst)
dmd_tform_mat = dmd_tform.params

dmd_affine_as_string = ''
dmd_nums = [str(i) for i in dmd_tform_mat.ravel()[:6]]
for i in dmd_nums[:6]:
    dmd_affine_as_string += i + ';'
    

In [ ]:
print(np.average(recorded_peaks[::2]))
print(np.average(recorded_peaks[1::2]))

In [ ]:
def binarize_img_thresh(input_img, thresh):
    #corrected_image = (input_img.astype(float) - dark_avg.astype(float))/flatfield_ch1.astype(float)
    corrected_image = input_img.astype(float)
    smth_corrected_image = filters.gaussian(corrected_image, 2, preserve_range = True)
    threshold = thresh
    binary_image = smth_corrected_image > threshold
    binary_image = segmentation.clear_border(binary_image)
    binary_image = morphology.remove_small_holes(binary_image, 200)
    binary_image = morphology.remove_small_objects(binary_image, 500)
    binary_image = morphology.binary_closing(binary_image, selem = morphology.disk(5))
    return(binary_image)

def run_detection_protocol(mode, channel_list, channel_id):
    core.set_config(mode, channel_list[channel_id])
    core.set_exposure(200)
    core.snap_image()
    tagged_image = core.get_tagged_image()
    h, w = tagged_image.tags['Height'], tagged_image.tags['Width']
    img = np.reshape(tagged_image.pix, newshape=[h, w])
    return(img)

def test_cells_present(input_img, thresh):
   
    binary_image = binarize_img_thresh(input_img, thresh)
    
    # ignore the 100 pixels arouund the border because they are not reliable
    D = 100
    binary_image[:D,:] = False
    binary_image[:,:D] = False
    binary_image[-D:,:] = False
    binary_image[:,-D:] = False
    
    lblim = measure.label(binary_image)
    reg = measure.regionprops(lblim, intensity_image = input_img)
   
    area_data = []
    polarization_data = []
    aspect_ratio_data = []
   
    filtered_bin_img = np.zeros_like(binary_image)
    
   
    if len(reg) > 0:
        
        for r in reg:
            yg, xg = r.centroid
            #print(xg, yg, r.area)
            yw, xw = r.weighted_centroid
            pol_magnitude = np.hypot(yg - yw, xg - xw)
            polarization_data.append(pol_magnitude)
            area_data.append(r.area)
            if r.minor_axis_length > 0:
                aspect_ratio_data.append(r.major_axis_length/r.minor_axis_length)
            else:
                aspect_ratio_data.append(np.nan)
        area_data = np.array(area_data)
        aspect_ratio_data = np.array(aspect_ratio_data)
        pass_test_conditions = (area_data > 1000) & (aspect_ratio_data < 5)

        if np.sum(pass_test_conditions) > 0:
            for i in np.where(pass_test_conditions)[0] + 1:
                filtered_bin_img[lblim == i] = 1
           
            return(True, filtered_bin_img)
             
        else:
            return(False, filtered_bin_img)
    else:
        return(False, filtered_bin_img)
   
   
def find_promising_cell(ch0_stack, ch1_stack, thresh):
    # find a consistently migrating cell
   
    candidate_found = False
    target_x = None
    target_y = None
   
    # parameters for identifying candidates:
    min_breathing_room = 0 # minimum distance (in pixels) between nearest cell (or edge of image)
    min_cell_persistence = 0.8 # minimum required "chemotactic index"
    min_cell_speed = 3 #minimum average frame-to-frame displacement of centroid (in pixels/frame)
    min_cell_area = 1500  # minimum cell size needed in pixels
    max_cell_area = 15000 # maximum cell size in pixels
    min_cell_ch0_sig = 600 # minimum average signal (prestimulus) in channel 0
    min_cell_ch1_sig = 0 # minimum average signal (prestimulus) in channel 1

   
    # set up a distance map to figure out how close a given coordinate is to the edge of the image
    edge = np.zeros((1024, 1024))
    edge[1:-1,1:-1] = 1
    edge_dist = distance_transform_edt(edge)
    
    bin_stack = []
    for t in range(len(ch1_stack)):
        input_img = ch1_stack[t]
        binary_img = binarize_img_thresh(input_img, thresh)
        bin_stack.append(binary_img)
        
    bin_stack = np.array(bin_stack)
    lbl_stack, num_potential_cells = measure.label(bin_stack, return_num = True)
    
    plt.figure(figsize = (8,8))
    plt.subplot(aspect = 'equal')
    cmap = cm.get_cmap('jet')
    cvec = [cmap(i) for i in np.linspace(0,1,len(bin_stack))]
    for i in range(len(bin_stack)):
        plt.contour(bin_stack[i], levels = [False], colors = [cvec[i]])
    plt.show()
    
    # if at least one cell is found, move on with analysis
    if np.sum(bin_stack[-1]) > 0:

        cell_position_x = []
        cell_position_y = []
        ch0_mean_signal = []
        ch1_mean_signal = []
        cell_areas = []
        cell_speed = []
        cell_persistence = []
        breathing_room = []

        for n in range(1, num_potential_cells + 1):

            if n in lbl_stack[-1]:
                isolated_cell = lbl_stack == n
                X_positions = []
                Y_positions = []
                sizes = []
                for t in range(len(isolated_cell)):
                    y, x = center_of_mass(isolated_cell[t])
                    X_positions.append(x)
                    Y_positions.append(y)
                    sizes.append(np.sum(isolated_cell[t]))

                cell_position_x.append(X_positions[-1])
                cell_position_y.append(Y_positions[-1])
                cell_areas.append(sizes[-1])
                ch0_mean_signal.append(np.average(ch0_stack[-1][isolated_cell[-1]]))
                ch1_mean_signal.append(np.average(ch1_stack[-1][isolated_cell[-1]]))

                path_length = np.sum(np.hypot(np.diff(X_positions), np.diff(Y_positions)))
                avg_speed = np.average(np.hypot(np.diff(X_positions), np.diff(Y_positions)))
                end_to_end_displacement = np.hypot(X_positions[-1] - X_positions[0], Y_positions[-1] - Y_positions[0])
                persistence_index = end_to_end_displacement/path_length

                cell_speed.append(avg_speed)
                cell_persistence.append(persistence_index)

        # if at least 2 cells are present:
        if len(cell_position_x) > 1:
            Dmat = spatial.distance.squareform(spatial.distance.pdist(np.vstack([cell_position_x, cell_position_y]).T))
            for i in range(len(Dmat)):
                closest_cell = np.min(np.delete(Dmat[i], i))
                closest_edge = edge_dist[int(cell_position_y[i]), int(cell_position_x[i])]

                breathing_room.append(np.min([closest_cell, closest_edge]))
               
        # if only 1 cell is present
        else:
            breathing_room.append(edge_dist[int(cell_position_y[0]), int(cell_position_x[0])])

        # convert to numpy arrays for easy element-wise calculations
        cell_position_x = np.array(cell_position_x)
        cell_position_y = np.array(cell_position_y)
        ch0_mean_signal = np.array(ch0_mean_signal)
        ch1_mean_signal = np.array(ch1_mean_signal)
        cell_areas = np.array(cell_areas)
        cell_speed = np.array(cell_speed)
        cell_persistence = np.array(cell_persistence)
        breathing_room = np.array(breathing_room)

        celldata = {'x': cell_position_x, 'y': cell_position_y, 'ch0_F': ch0_mean_signal, 'ch1_F': ch1_mean_signal, 'Area': cell_areas, 'Speed': cell_speed, 'Persistence': cell_persistence, 'BreathingRoom': breathing_room}
        df = pd.DataFrame.from_dict(celldata)
        print(df)

        # test all identified cells to see if they meet minimum requirements as outlined above
        # if more than one is found, identify the one with the most "breathing room"
        cond1 = (breathing_room > min_breathing_room)
        cond2 = (cell_persistence > min_cell_persistence)
        cond3 = (cell_speed > min_cell_speed)
        cond4 = (cell_areas > min_cell_area)
        cond5 = (cell_areas < max_cell_area)
        cond6 = (ch0_mean_signal > min_cell_ch0_sig)
        cond7 = (ch1_mean_signal > min_cell_ch1_sig)
        min_requirements = (cond1 & cond2 & cond3 & cond4 & cond5 & cond6 & cond7)

       
        if np.sum(min_requirements) > 0:
            target_x = cell_position_x[min_requirements][np.argmax(breathing_room[min_requirements])]
            target_y = cell_position_y[min_requirements][np.argmax(breathing_room[min_requirements])]
            candidate_found = True
        else:
            candidate_found = False

    else:
        candidate_found = False
       
    return(candidate_found, target_x, target_y)



def run_migration_test(mode, channel_list, num_imgs, wait_interval):

    ch0_stack = []
    ch1_stack = []

    for repeat in range(num_imgs):
        start_time = time.time()

        core.set_config(mode, channel_list[0])
        core.set_exposure(200)
        core.snap_image()
        tagged_image = core.get_tagged_image()
        h, w = tagged_image.tags['Height'], tagged_image.tags['Width']
        pixels = np.reshape(tagged_image.pix, newshape=[h, w])
        ch0_stack.append(pixels)

        core.set_config(mode, channel_list[1])
        core.set_exposure(200)
        core.snap_image()
        tagged_image = core.get_tagged_image()
        h, w = tagged_image.tags['Height'], tagged_image.tags['Width']
        pixels = np.reshape(tagged_image.pix, newshape=[h, w])
        ch1_stack.append(pixels)

        end_time = time.time()
        
        remaining_time = wait_interval - (end_time - start_time)
        if remaining_time > 0:
            time.sleep(remaining_time)
       
    return(np.array(ch0_stack), np.array(ch1_stack))


def make_data_directory(folder_base_name):
    # create a storage folder & metadata file
    folder_number = len([i for i in os.listdir("C:\\Data\\Jason\\python_controlled_acq\\") if i[:-4] == folder_base_name])
    folder_path = "C:\\Data\\Jason\\python_controlled_acq\\" + folder_base_name + '_' + str(folder_number).zfill(3)
    while not os.path.exists(folder_path):
        try:
            os.mkdir(folder_path)
        except:
            print('Directory conflict, trying another folder name')
            folder_number += 1
            folder_path = "C:\\Data\\Jason\\python_controlled_acq\\" + folder_base_name + '_' + str(folder_number).zfill(3)
    return(folder_path)

In [ ]:
print('x', core.get_x_position())
print('y', core.get_y_position())
print('z', core.get_position())
print('exposure_ms', core.get_exposure())
affine_matrix = np.vstack([np.array([float(i) for i in core.get_pixel_size_affine_as_string().split(';')]).reshape(2, 3), [0,0,1]])


aft_im_to_stage = transform.AffineTransform(affine_matrix)
aft_dmd_to_im = transform.AffineTransform(dmd_tform_mat)

print(affine_matrix)
print(dmd_tform_mat)

dmd_scalar_factor = np.average(aft_dmd_to_im.scale)

In [ ]:
# collect current coords
# MANUALLY NAVIGATE TO CENTER OF WELL
center_y = core.get_y_position()
center_x = core.get_x_position()

print(center_x, center_y)

In [ ]:
# generate a circular-shaped set of imaging stage coordinates 
R = 7

rr, cc = draw.disk((0, 0), R)
unique_rs = np.sort(np.unique(rr))

rr_snake = []
cc_snake = []

flip = -1
for i in unique_rs:
    for j in cc[rr == i]:
        rr_snake.append(i)
        cc_snake.append(flip * j)
    flip *= -1

relative_y = 200 * np.array(cc_snake)
relative_x = 200 * np.array(rr_snake)
absolute_x = center_x + np.array(relative_x)
absolute_y = center_y + np.array(relative_y)
position_num_ids = range(len(absolute_x))
print(f'Set up to image {len(position_num_ids)} points\n')


plt.figure(figsize = (8,8))
plt.subplot(aspect = 'equal')
plt.plot(absolute_x, absolute_y, color = 'k', ls = '--')
plt.scatter(absolute_x, absolute_y)

theta = np.linspace(0, 2 * math.pi, 100)
plt.plot(3200 * np.cos(theta) + center_x, 3200 * np.sin(theta) + center_y)

plt.show()

In [ ]:

DMD_CENTER_X = 740
DMD_CENTER_Y = 600


pattern = np.zeros((600,800), dtype = np.uint8)
np_to_dmd(pattern)
v_650(s1, 0)

NUM_REPEATS = 3

num_images_to_acquire = 60
wait_time_interval = 6
folder_base_name = '220314_KWC_35-52-97_simpleStimScreen'
mode = '-TIRF'
channel_list = ["561-488LP", "640-488LP", "REFLECT-488LP"]

zlevel = 0
track_cell = True
segmentation_channel = 1
intensity_thresh = 650

# set up ROI with side length 2*D
D = 200
ROI_xmin = DMD_CENTER_X - D
ROI_xmax = DMD_CENTER_X + D
ROI_ymin = DMD_CENTER_Y - D
ROI_ymax = DMD_CENTER_Y + D



for repeat in range(NUM_REPEATS):
    
    for i in position_num_ids:
        x_R = np.nan
        y_R = np.nan
        x_L = np.nan
        y_L = np.nan

        # move to position
        print('Position' + str(i + 1).zfill(3) + ' of ' + str(len(position_num_ids)).zfill(3))
        core.set_xy_position(np.double(absolute_x[i]), np.double(absolute_y[i]))
        core.wait_for_system()
        time.sleep(1)

        # run cell detection protocol
        detection_image = run_detection_protocol(mode, channel_list, segmentation_channel)
        cells_present_condition, segmented_proposed_cells = test_cells_present(detection_image, intensity_thresh)

        # display info for user
        plt.figure(figsize = (16,8))

        # current_position
        plt.subplot(121, aspect = 'equal')
        plt.plot(absolute_x, absolute_y, color = 'k', ls = '--', zorder = 0)
        plt.scatter(absolute_x, absolute_y, zorder = 1)
        plt.scatter(absolute_x[i], absolute_y[i], s = 200, lw = 2, ec = 'k', facecolor = 'None')
        
        # image and segmentation
        plt.subplot(122)
        plt.imshow(detection_image)
        plt.colorbar()
        plt.contour(segmented_proposed_cells, levels = [False], colors = 'w', linestyles = '--')
        plt.title('Cell Presence Test')
        plt.show()

        # if preliminary test was passed, run secondary test
        if cells_present_condition:

            print('Cell likely present, starting migration test')
            
            
            ch0_data, ch1_data = run_migration_test(mode, channel_list, num_imgs = 5, wait_interval = 5) #DEFAULT IS 5, 5
            continue_imaging, x_in_img, y_in_img = find_promising_cell(ch0_data, ch1_data, intensity_thresh)

            plt.figure(figsize = (8,8))
            plt.imshow(np.max(ch1_data, axis = 0))
            plt.scatter(x_in_img, y_in_img, facecolor = 'None', ec = 'w', s = 500)
            plt.title('Max Projection over Time')
            plt.show()

            stage_x_sequence = []
            stage_y_sequence = []
            
            if continue_imaging:
                print('Migration test passed! Moving on to opto experiment')
                
                # move promising cell roughly to center of dmd field
                move_from = aft_im_to_stage([x_in_img, y_in_img])[0]
                move_to = aft_im_to_stage([DMD_CENTER_X, DMD_CENTER_Y])[0]
                DX, DY = move_from - move_to
                new_x = core.get_x_position() + DX
                new_y = core.get_y_position() + DY
                core.set_xy_position(new_x, new_y)
                
                # determine assay type
                brightness = 50000 #np.random.choice([0, 15000, 30000, 45000])
                variable_options = ['none', 'global', 'front', 'side', 'back', 'lateral90', 'lateral45']
                variable_choice = len(os.listdir("C:\\Data\\Jason\\python_controlled_acq\\"))%len(variable_options)
                assay_choice = variable_options[variable_choice]
                print(f'Running {assay_choice}-type assay')
                folder_path = make_data_directory(folder_base_name + '_' + assay_choice + '_' + str(brightness) + '_Intensity')
                
                ch0_stack = []
                bin_stack = []
                lght_bin_stack = []

                experiment_start_time = time.time()
                
                lost_cell_count = 0
                
                continuous_imaging = True
                for t in range(num_images_to_acquire):
                    
                    if lost_cell_count <= 3:

                        start_time = time.time()
                        description_dict = {'status': 'observing'}

                        # collect images in sequence (separated from other processing for the sake of speed)
                        tiff_collection = []
                        md_collection = []
                        for channel in range(len(channel_list)):
                            core.set_config(mode, channel_list[channel])
                            core.set_exposure(200)
                            core.snap_image()
                            tagged_image = core.get_tagged_image()
                            metadata_dict = tagged_image.tags
                            metadata_dict['elapsed_time_s'] = str(time.time() - experiment_start_time)
                            metadata_dict['stage_x_pos'] = str(core.get_x_position())
                            metadata_dict['stage_y_pos'] = str(core.get_y_position())
                            metadata_dict['dmd_affine_transform'] = dmd_affine_as_string
                            metadata_dict['roi_xmin_xmax_ymin_ymax'] = str(ROI_xmin) + ';' + str(ROI_xmax) + ';' + str(ROI_ymin) + ';' + str(ROI_ymax)
                            tiff_collection.append(tagged_image)
                            md_collection.append(metadata_dict)

                        # now save those images and run analysis
                        for channel in range(len(channel_list)):
                            img = tiff_collection[channel]
                            metadata_dict = img.tags
                            h, w = metadata_dict['Height'], metadata_dict['Width']
                            pixels = np.reshape(img.pix, newshape=[h, w])

                            image_dest = folder_path + f"\\img_channel{str(channel).zfill(3)}_position{str(i).zfill(3)}_time{str(t).zfill(9)}_z{str(zlevel).zfill(3)}.tif"
                            tifffile.imwrite(image_dest, pixels[ROI_ymin:ROI_ymax, ROI_xmin:ROI_xmax], metadata = md_collection[channel])

                            if channel == segmentation_channel:

                                ch0_stack.append(pixels)

                                binary_image = binarize_img_thresh(pixels, intensity_thresh)

                                isolated_cell = get_closest_cell(binary_image, DMD_CENTER_X, DMD_CENTER_Y)

                                if np.sum(isolated_cell) > 0:
                                    possible_y_center, possible_x_center = center_of_mass(isolated_cell)
                                    dist_from_target = np.hypot(possible_y_center - DMD_CENTER_Y, possible_x_center - DMD_CENTER_X)
                                    if dist_from_target > 100:
                                        isolated_cell = np.zeros_like(isolated_cell)

                                bin_stack.append(isolated_cell)

                                # update optogenetic stimulation status
                                if (t >= 10):

                                    # calculate angle for opto stim
                                    if track_cell:
                                        dx_track = np.average(np.diff(stage_x_sequence[:10]))
                                        dy_track = -np.average(np.diff(stage_y_sequence[:10])) # y sign is flipped between image and stage coords
                                        theta = np.arctan2(dy_track, dx_track)

                                    else:
                                        theta = get_move_polar_angle(bin_stack[7], bin_stack[10])

                                    # set brightness
                                    brightness = 50000
                                    pattern = np.zeros((600,800))

                                    # check if cell is being tracked and update points if possible
                                    if np.sum(bin_stack[-1]) > 0:
                                        
                                        if assay_choice == 'none':
                                            pass
                                        
                                        elif assay_choice == 'global':
                                            pattern = np.ones((600,800))
                                            
                                        elif assay_choice == 'front':
                                            x_R, y_R, x_L, y_L = get_lateral_points(bin_stack[-1], theta - math.pi/2) # -math.pi/2 for front, +math.pi/2 for back
                                            x_in_dmd, y_in_dmd = dmd_from_img(x_L, y_L)
                                            rr_disk, cc_disk = draw.disk((y_in_dmd, x_in_dmd), 20, shape = (600, 800))
                                            pattern[rr_disk, cc_disk] = 1
                                            
                                        elif assay_choice == 'side':
                                            x_R, y_R, x_L, y_L = get_lateral_points(bin_stack[-1], theta) # -math.pi/2 for front, +math.pi/2 for back
                                            x_in_dmd, y_in_dmd = dmd_from_img(x_L, y_L)
                                            rr_disk, cc_disk = draw.disk((y_in_dmd, x_in_dmd), 20, shape = (600, 800))
                                            pattern[rr_disk, cc_disk] = 1
                                            
                                        elif assay_choice == 'back':
                                            x_R, y_R, x_L, y_L = get_lateral_points(bin_stack[-1], theta + math.pi/2) # -math.pi/2 for front, +math.pi/2 for back
                                            x_in_dmd, y_in_dmd = dmd_from_img(x_L, y_L)
                                            rr_disk, cc_disk = draw.disk((y_in_dmd, x_in_dmd), 20, shape = (600, 800))
                                            pattern[rr_disk, cc_disk] = 1
                                            
                                        elif assay_choice == 'lateral90':
                                            x_R, y_R, x_L, y_L = get_lateral_points(bin_stack[-1], theta) # -math.pi/2 for front, +math.pi/2 for back
                                            x_in_dmd, y_in_dmd = dmd_from_img(x_L, y_L)
                                            rr_disk, cc_disk = draw.disk((y_in_dmd, x_in_dmd), 20, shape = (600, 800))
                                            pattern[rr_disk, cc_disk] = 1
                                            x_in_dmd, y_in_dmd = dmd_from_img(x_R, y_R)
                                            rr_disk, cc_disk = draw.disk((y_in_dmd, x_in_dmd), 20, shape = (600, 800))
                                            pattern[rr_disk, cc_disk] = 1
                                            
                                        elif assay_choice == 'lateral45':
                                            x_R, y_R, x_L, y_L = get_lateral_points(bin_stack[-1], theta - math.pi/2 + math.pi/4) # -math.pi/2 for front, +math.pi/2 for back
                                            x_in_dmd, y_in_dmd = dmd_from_img(x_L, y_L)
                                            rr_disk, cc_disk = draw.disk((y_in_dmd, x_in_dmd), 20, shape = (600, 800))
                                            pattern[rr_disk, cc_disk] = 1
                                            x_R, y_R, x_L, y_L = get_lateral_points(bin_stack[-1], theta - math.pi/2 - math.pi/4)
                                            x_in_dmd, y_in_dmd = dmd_from_img(x_L, y_L)
                                            rr_disk, cc_disk = draw.disk((y_in_dmd, x_in_dmd), 20, shape = (600, 800))
                                            pattern[rr_disk, cc_disk] = 1

                                        np_to_dmd(pattern)
                                        v_650(s1, brightness)
                                        
                                        status = 'tracking cell'
                                        lost_cell_count = 0
                                        
                                    else:
                                        status = 'lost cell'
                                        lost_cell_count += 1
                                        

                                    R = np.round(dmd_scalar_factor * 20, 1)
                                    et = np.round(time.time() - experiment_start_time, 3)
                                    description_dict = {'status': status, 'assay': assay_choice, 'theta': theta, 'Intensity': str(brightness),'image_x': np.round(x_R, 2),'image_y': np.round(y_R, 2), 'image_r': R, 'elapsed_time': et}
                                    print(description_dict)



                        # save binary image and predicted dmd binary image
                        bin_im_path = folder_path + f"\\img_channel{str(len(channel_list)).zfill(3)}_position{str(i).zfill(3)}_time{str(t).zfill(9)}_z{str(zlevel).zfill(3)}.tif"
                        tifffile.imwrite(bin_im_path, isolated_cell.astype(np.int16)[ROI_ymin:ROI_ymax, ROI_xmin:ROI_xmax])

                        stage_x_sequence.append(core.get_x_position())
                        stage_y_sequence.append(core.get_y_position())

                        dmd_im_path = folder_path + f"\\img_channel{str(len(channel_list) + 1).zfill(3)}_position{str(i).zfill(3)}_time{str(t).zfill(9)}_z{str(zlevel).zfill(3)}.tif"
                        dmd_im = predict_DMD_image()
                        tifffile.imwrite(dmd_im_path, dmd_im.astype(np.int16)[ROI_ymin:ROI_ymax, ROI_xmin:ROI_xmax], metadata = description_dict)
                        lght_bin_stack.append(dmd_im)

                        end_time = time.time()
                        remaining_time = wait_time_interval - (end_time - start_time)
                        if remaining_time > 0:
                            time.sleep(remaining_time)

                        # if tracking, move cell just before the loop starts over
                        if track_cell:    

                            current_cell_pos_y, current_cell_pos_x = center_of_mass(isolated_cell)
                            move_from = aft_im_to_stage([current_cell_pos_x, current_cell_pos_y])[0]
                            move_to = aft_im_to_stage([DMD_CENTER_X, DMD_CENTER_Y])[0]
                            DX, DY = move_from - move_to
                            if np.hypot(DX, DY) < 100:
                                new_x = core.get_x_position() + DX
                                new_y = core.get_y_position() + DY
                                core.set_xy_position(new_x, new_y)
                    else:
                        print('Lost cell, removing data')
                        try:
                            shutil.rmtree(folder_path)
                            continuous_imaging = False
                        except OSError as e:
                            print("Error: %s - %s." % (e.filename, e.strerror))
                
                if continuous_imaging:
                    # verify presence of opto tools in blue and green channels
                    core.set_config('-TIRF', '488')
                    core.set_exposure(200)
                    core.snap_image()
                    tagged_image = core.get_tagged_image()
                    metadata_dict = tagged_image.tags
                    metadata_dict['elapsed_time_s'] = str(time.time() - experiment_start_time)
                    metadata_dict['stage_x_pos'] = str(core.get_x_position())
                    metadata_dict['stage_y_pos'] = str(core.get_y_position())
                    metadata_dict['dmd_affine_transform'] = dmd_affine_as_string
                    metadata_dict['roi_xmin_xmax_ymin_ymax'] = str(ROI_xmin) + ';' + str(ROI_xmax) + ';' + str(ROI_ymin) + ';' + str(ROI_ymax)
                    h, w = metadata_dict['Height'], metadata_dict['Width']
                    pixels = np.reshape(tagged_image.pix, newshape=[h, w])
                    image_dest = folder_path + f"\\img_channel{str(len(channel_list) + 2).zfill(3)}_position{str(i).zfill(3)}_time{str(t).zfill(9)}_z{str(zlevel).zfill(3)}.tif"
                    tifffile.imwrite(image_dest, pixels[ROI_ymin:ROI_ymax, ROI_xmin:ROI_xmax], metadata = md_collection[channel])

                    core.set_config('-TIRF', '405')
                    core.set_exposure(200)
                    core.snap_image()
                    tagged_image = core.get_tagged_image()
                    metadata_dict = tagged_image.tags
                    metadata_dict['elapsed_time_s'] = str(time.time() - experiment_start_time)
                    metadata_dict['stage_x_pos'] = str(core.get_x_position())
                    metadata_dict['stage_y_pos'] = str(core.get_y_position())
                    metadata_dict['dmd_affine_transform'] = dmd_affine_as_string
                    metadata_dict['roi_xmin_xmax_ymin_ymax'] = str(ROI_xmin) + ';' + str(ROI_xmax) + ';' + str(ROI_ymin) + ';' + str(ROI_ymax)
                    h, w = metadata_dict['Height'], metadata_dict['Width']
                    pixels = np.reshape(tagged_image.pix, newshape=[h, w])
                    image_dest = folder_path + f"\\img_channel{str(len(channel_list) + 3).zfill(3)}_position{str(i).zfill(3)}_time{str(t).zfill(9)}_z{str(zlevel).zfill(3)}.tif"
                    tifffile.imwrite(image_dest, pixels[ROI_ymin:ROI_ymax, ROI_xmin:ROI_xmax], metadata = md_collection[channel])


                
                # turn off lights and pattern for fresh restart
                v_650(s1, 0)
                pattern = np.zeros((600,800), dtype = np.uint8)
                np_to_dmd(pattern)

                # display for user
                if track_cell:
                    stage_x_sequence = np.array(stage_x_sequence)
                    stage_y_sequence = np.array(stage_y_sequence)

                    plt.figure(figsize = (6,6))
                    plt.subplot(aspect = 'equal')
                    plt.plot(stage_x_sequence - stage_x_sequence[0], stage_y_sequence - stage_y_sequence[0])
                    plt.title(f'Position: {position_num_ids[i]}')
                    plt.show()


                    plt.figure(figsize = (8,8))
                    plt.subplot(aspect = 'equal')
                    cmap = cm.get_cmap('jet')
                    cvec = [cmap(timestep) for timestep in np.linspace(0,1,len(bin_stack))]

                    for i in range(len(bin_stack)):
                        for ctrs in measure.find_contours(bin_stack[i], level = False):
                            y_segment_imcoords, x_segment_imcoords = ctrs.T
                            y_segment_stgcoords, x_segment_stgcoords = aft_im_to_stage(np.array([y_segment_imcoords, x_segment_imcoords]).T).T

                            x_translated = -x_segment_stgcoords + stage_x_sequence[i] - stage_x_sequence[0]
                            y_translated = -y_segment_stgcoords + stage_y_sequence[i] - stage_y_sequence[0]

                            plt.plot(x_translated, y_translated, color = cvec[i])

                        for ctrs in measure.find_contours(lght_bin_stack[i], level = False):
                            y_segment_imcoords, x_segment_imcoords = ctrs.T
                            y_segment_stgcoords, x_segment_stgcoords = aft_im_to_stage(np.array([y_segment_imcoords, x_segment_imcoords]).T).T

                            x_translated = -x_segment_stgcoords + stage_x_sequence[i] - stage_x_sequence[0]
                            y_translated = -y_segment_stgcoords + stage_y_sequence[i] - stage_y_sequence[0]

                            plt.plot(x_translated, y_translated, color = cvec[i], ls = '--')

                    plt.show()

            else:
                print('Likely no migrating cells here')
        else:
            print('Likely no cells here')

        print()
        